# Linear Regression 실습

이번 실습에서는 linear regression에 대한 gradient descent를 직접 구현해봅니다. 여기서 사용할 문제들은 크게 두 가지로 OR 문제와 XOR 문제입니다.

먼저 필요한 library들을 import합시다.

In [1]:
import torch
import numpy as np

In [2]:
from torch import Tensor
from typing import List, Tuple, Dict, Any, Union

## NOTE

텐서로 정의되는 2차원이 4개 들어가있는 x 한개를 (4,2) 라고 부른다?
아님 2차원 데이터 4개?를 표현한 것?

### randn 의 정의

randn 은 표준 정규 분포에서 랜덤 샘플을 추출하는 함수이다

In [3]:

# 1차원 텐서 생성 (길이 4)
x = torch.randn(4)
# 결과 예시: tensor([-2.1436, 0.9966, 2.3426, -0.6366])

# 2차원 텐서 생성 (2행 3열)
y = torch.randn(2, 3)
# 결과 예시: tensor([[ 1.5954, 2.8929, -1.0923],
#                   [ 1.1719, -0.4709, -0.1996]])

## OR Problem

OR은 0 또는 1의 값을 가질 수 있는 두 개의 정수를 입력으로 받아 둘 중에 하나라도 1이면 1을 출력하고 아니면 0을 출력하는 문제입니다.
즉, 우리가 학습하고자 하는 함수는 2개의 정수를 입력받아 하나의 정수를 출력하면됩니다. 이러한 함수를 학습하기 위한 data는 다음과 같이 구성할 수 있습니다.

## NOTE

텐서를 정의하는 개념에 대해 설명한다
우선 하나의 텐서에 대한 정의로 n차원이든 정의된다

그리고 그 크기에 대해 사이즈를 아래 예시의 2차원 백터 4 개는 shape를 봤을 떄 [4,2] 처럼 정의된다
여기서 나의 해석 방법과 텐서의 해석 방법이 다른데
텐서는 데이터의 개수를 먼저 정의하고 그 뒤에 데이터의 차원을 세고 있는 것을 알 수 있다

```
x = torch.tensor([
    [0., 0.],
    [0., 1.],
    [1., 0.],
    [1., 1.]
]) 
```
에 대해 해석할 때

해석하면 다음과 같다 일단 텐서에는 4개의 A 텐서가 있다 > A는 벡터를 담고 있는 텐서이다?
이러한 텐서의 정의를 통해 우리는 데이터의 개수와 데이터의 차원을 명확하게 알 수 있고 이를 통해 우리는 데이터의 형태를 파악할 수 있다

아래의 x와 y는 입력 데이터와 출력 데이터다

or이기 때문에 정답이 0,1,1,1 인 것

In [4]:
x = torch.tensor([
    [0., 0.],
    [0., 1.],
    [1., 0.],
    [1., 1.]
])
y = torch.tensor([0, 1, 1, 1])

print(x.shape, y.shape)


torch.Size([4, 2]) torch.Size([4])


출력 결과에서 볼 수 있다시피 $x$의 shape은 (4, 2)로, 총 4개의 two-dimensional data 임을 알 수 있습니다. $y$는 각 $x_i$에 대한 label로 우리가 설정한 문제의 조건을 잘 따라가는 것을 알 수 있습니다.

다음으로는 linear regression의 parameter들인 $w, b$를 정의하겠습니다.

In [5]:
w = torch.randn((1, 2))
b = torch.randn((1, 1))

print(w.shape, b.shape)
print(w, b)



torch.Size([1, 2]) torch.Size([1, 1])
tensor([[ 1.3616, -0.1143]]) tensor([[0.4757]])


$w$는 1x2의 벡터이고 $b$는 1x1의 scalar임을 알 수 있습니다. 여기서는 `torch.randn`을 사용하여 standard normal distribution을 가지고 초기화하였습니다.

이러한 $w, b$와 data $x, y$가 주어졌을 때 우리가 학습한 $w, b$의 성능을 평가하는 함수를 구현합시다.
평가 함수는 다음과 같이 MSE로 정의됩니다:
$$l(f) := MSE(f(x), y) = \frac{1}{n} \sum_{i=1}^n (f(x_i) - y)^2.$$
이를 구현한 코드는 다음과 같습니다.

## NOTE

여기서 w는 전체 영역의 크기에 대해 정의하고 있고
b는 한개의 스칼라를 정의하고 있음을 알 수 있다

pred 는 결과 출력할 때 사용한 함수임

w는 전체 영역의 현재 가중치
x.T 는 행렬 계산을 위해 순서를 맞추기 위한 전치

b는 현재 입력 값에 대해 추가할 편향 값


x.T는 한 번에 모든 데이터 샘플에 대한 계산을 수행하기 위해 행렬의 축을 재정렬하는 것

크기는 작지만 x는 데이터 입력 라고 할 수 있음

이것이 바로 효율적인 벡터화 연산의 핵심으로, 딥러닝에서 배치 처리를 할 때 굉장히 중요한 기법 
한 번의 연산으로 모든 샘플에 대한 계산을 수행하므로 개별적으로 처리하는 것보다 훨씬 빠르다

In [6]:
def pred(w, b, x):
  print('pred:',w, b, x ,x.T)
  return torch.matmul(w, x.T) + b


def loss(w, b, x, y):
  return (y - pred(w, b, x)).pow(2).mean()

먼저 `def pred(w, b, x)`는 $wx^T + b$, 즉 1차 함수 $f$의 $x$에 대한 결과를 반환하는 함수를 구현했습니다.
이를 이용하여 주어진 label $y$와의 MSE를 측정하는 코드가 `def loss(w, b, x, y)`에 구현되어있습니다.

다음은 MSE를 기반으로 $w, b$의 gradient를 구하는 코드를 구현하겠습니다.
MSE에 대한 $w$의 gradient는 다음과 같이 구할 수 있습니다:
$$\frac{\partial l}{\partial w} = \frac{1}{n} \sum_{i=1}^n 2(wx_i^T + b - y)x_i.$$
$b$에 대한 gradient는 다음과 같습니다:
$$\frac{\partial l}{\partial b} = \frac{1}{n} \sum_{i=1}^n 2(wx_i^T + b - y).$$
이를 코드로 구현하면 다음과 같습니다.

In [7]:
def grad_w(w, b, x, y):
  # w: (1, d), b: (1, 1), x: (n, d), y: (n)
  tmp1 = torch.matmul(w, x.T)  # (1, n)
  tmp2 = tmp1 + b              # (1, n)
  tmp3 = 2 * (tmp2 - y[None])  # (1, n)
  grad_item = tmp3.T * x       # (n, d)
  return grad_item.mean(dim=0, keepdim=True)  # (1, d)


def grad_b(w, b, x, y):
  # w: (1, d), b: (1, 1), x: (n, d), y: (n)
  grad_item = 2 * (torch.matmul(w, x.T) + b - y[None])  # (1, n)
  return grad_item.mean(dim=-1, keepdim=True)           # (1, 1)

여기서 중요한 것은 shape에 맞춰서 연산을 잘 사용해야한다는 것입니다. Shape과 관련된 설명은 `[Chapter 0]`의 Numpy에서 설명했으니, 복습하신다는 느낌으로 주석으로 써놓은 shape들을 유도해보시면 좋을 것 같습니다. 중요한 것은 반환되는 tensor의 shape이 우리가 구하고자 하는 gradient와 일치해야 한다는 것입니다. 예를 들어 $w$의 $l$에 대한 gradient는 $w$와 shape이 동일해야 합니다.

마지막으로 gradient descent 함수를 구현하겠습니다. Gradient descent는 다음과 같이 정의됩니다:
$$w^{(new)} = w^{(old)} - \eta \frac{\partial l}{\partial w} \biggr\rvert_{w = w^{(old)}}.$$
Gradient는 위에서 구현했으니 이를 활용하여 learning rate $\eta$가 주어졌을 때 $w, b$를 update하는 코드를 구현할 수 있습니다. 구현한 결과는 다음과 같습니다.

In [8]:
def update(x, y, w, b, lr):
  w = w - lr * grad_w(w, b, x, y)
  b = b - lr * grad_b(w, b, x, y)
  return w, b

Gradient descent(경사 하강법)에 해당하는 코드는 모두 구현하였습니다. 이제 학습하는 코드를 구현하겠습니다:

In [9]:
def train(n_epochs, lr, w, b, x, y):
  for e in range(n_epochs):
    w, b = update(x, y, w, b, lr)
    print(f"Epoch {e:3d} | Loss: {loss(w, b, x, y)}")
  return w, b

여기서 `n_epochs`는 update를 하는 횟수를 의미합니다. 매 update 이후에 `loss` 함수를 사용하여 잘 수렴하고 있는지 살펴봅니다. 실제로 이 함수를 실행한 결과는 다음과 같습니다.

In [10]:
n_epochs = 100
lr = 0.1

w, b = train(n_epochs, lr, w, b, x, y)
print(w, b)

pred: tensor([[ 1.2836, -0.1185]]) tensor([[0.4140]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   0 | Loss: 0.3724440932273865
pred: tensor([[ 1.2198, -0.1122]]) tensor([[0.3705]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   1 | Loss: 0.31607845425605774
pred: tensor([[ 1.1664, -0.0990]]) tensor([[0.3396]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   2 | Loss: 0.27842044830322266
pred: tensor([[ 1.1207, -0.0814]]) tensor([[0.3178]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   3 | Loss: 0.25097325444221497
pred: tensor([[ 1.0809, -0.0611]]) tensor([[0.3022]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) te

잘 수렴하는 것을 확인하였습니다. 마지막으로 OR data에 대한 $w, b$의 예측 결과와 label을 비교해봅시다.

In [11]:
print(pred(w, b, x))
print(y)

pred: tensor([[0.5038, 0.4951]]) tensor([[0.2506]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
tensor([[0.2506, 0.7457, 0.7545, 1.2495]])
tensor([0, 1, 1, 1])


아래 값으로 수렴하는 것을 볼 수 있음

tensor([[0.2500, 0.7500, 0.7500, 1.2500]])
tensor([0, 1, 1, 1])

예측 결과를 볼 수 있다시피 우리의 linear regression model은 0과 1에 해당하는 data를 잘 구분하는 것을 알 수 있습니다.

# XOR Problem

이번에는 XOR를 학습해보겠습니다. XOR은 OR과 똑같은 입력을 받는 문제로, 두 개의 0 또는 1의 정수가 들어왔을 때 두 정수가 다르면 1, 아니면 0을 출력해야 합니다.
먼저 data를 만들어보겠습니다:

In [12]:
# x = torch.tensor([
#     [0., 0.],
#     [0., 1.],
#     [1., 0.],
#     [1., 1.]
# ])
# y = torch.tensor([0, 1, 1, 0])

print(x.shape, y.shape)

torch.Size([4, 2]) torch.Size([4])


보시다시피 shape이나 생성 과정은 OR과 똑같습니다. 다른 것은 $y$에서의 labeling입니다. OR과 다르게 $x = (1, 1)$에 대해서는 0을 labeling했습니다.
이러한 사소한 차이에 대해서도 linear regression model이 잘 학습할 수 있을지 살펴보겠습니다.

## NOTE
여러번 돌려보면서 w,b 가  수렴하는 것을 볼 수 있음

그런데 그것과 별개로 loss가 0.25로 수렴하는 것도 확인 할 수 있는데
왜 그렇게 되는지 의문임

In [13]:
n_epochs = 100
lr = 0.1

w, b = train(n_epochs, lr, w, b, x, y)
print(w, b)

pred: tensor([[0.5036, 0.4953]]) tensor([[0.2506]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   0 | Loss: 0.06250877678394318
pred: tensor([[0.5034, 0.4955]]) tensor([[0.2506]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   1 | Loss: 0.06250791996717453
pred: tensor([[0.5032, 0.4958]]) tensor([[0.2506]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   2 | Loss: 0.06250714510679245
pred: tensor([[0.5031, 0.4960]]) tensor([[0.2506]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch   3 | Loss: 0.06250645965337753
pred: tensor([[0.5029, 0.4962]]) tensor([[0.2505]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0.

tensor([[0.2503]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch  18 | Loss: 0.06250140070915222
pred: tensor([[0.5013, 0.4982]]) tensor([[0.2503]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch  19 | Loss: 0.06250126659870148
pred: tensor([[0.5012, 0.4982]]) tensor([[0.2503]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch  20 | Loss: 0.06250114738941193
pred: tensor([[0.5012, 0.4983]]) tensor([[0.2503]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
Epoch  21 | Loss: 0.06250104308128357
pred: tensor([[0.5011, 0.4984]]) tensor([[0.2503]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0

이전과는 다르게 loss가 1.0보다 작아지지 않는 것을 알 수 있습니다. 실제 예측 결과를 살펴보면 다음과 같습니다.

In [14]:
print(pred(w, b, x))
print(y)

pred: tensor([[0.5000, 0.5000]]) tensor([[0.2500]]) tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]) tensor([[0., 0., 1., 1.],
        [0., 1., 0., 1.]])
tensor([[0.2500, 0.7500, 0.7500, 1.2500]])
tensor([0, 1, 1, 1])


보시다시피 0과 1에 해당하는 data들을 잘 구분하지 못하는 모습니다. Linear regression model은 XOR을 잘 처리하지 못하는 것을 우리는 이번 실습을 통해 알 수 있습니다.